Отношения: продукты, категории и их связи.

Каждому продукту может соответствовать несколько категорий или ни одной.

А каждой категории может соответствовать несколько продуктов или ни одного.

In [ ]:
!pip install findspark
!pip install pyspark

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3099, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info. Did you mean: 'egg_info'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py",

In [ ]:
from pyspark.sql import SparkSession, Row

# Создание сессии Spark
spark = SparkSession.builder.appName("product").getOrCreate()

# Создание DataFrame для продуктов
product_data = [
    Row(product_id=1, product_name='Laptop'),
    Row(product_id=2, product_name='Mouse'),
    Row(product_id=3, product_name='Keyboard'),
    Row(product_id=4, product_name='Monitor'),
    Row(product_id = 5, product_name='CPU')
]
product_df = spark.createDataFrame(product_data)

# Создание DataFrame для категорий
category_data = [
    Row(category_id=1, category_name='Electronics'),
    Row(category_id=2, category_name='Computer Accessories'),
    Row(category_id=3, category_name='Peripherals')
]
category_df = spark.createDataFrame(category_data)

# Создание DataFrame для связей между продуктами и категориями
product_category_data = [
    Row(product_id=1, category_id=1),  # Laptop - Electronics
    Row(product_id=2, category_id=2),  # Mouse - Computer Accessories
    Row(product_id=3, category_id=2),  # Keyboard - Computer Accessories
    Row(product_id=4, category_id=1),  # Monitor - Electronics
    Row(product_id=4, category_id=3)   # Monitor - Peripherals
]
product_category_df = spark.createDataFrame(product_category_data)

# Вывод созданных DataFrame
product_df.show()
category_df.show()
product_category_df.show()


+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|      Laptop|
|         2|       Mouse|
|         3|    Keyboard|
|         4|     Monitor|
|         5|         CPU|
+----------+------------+

+-----------+--------------------+
|category_id|       category_name|
+-----------+--------------------+
|          1|         Electronics|
|          2|Computer Accessories|
|          3|         Peripherals|
+-----------+--------------------+

+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          2|
|         3|          2|
|         4|          1|
|         4|          3|
+----------+-----------+



In [ ]:
def product_cat(product_df, product_category_df, category_df):
  product_category_pairs = product_df.join(product_category_df, 'product_id', how='left') \
    .join(category_df, 'category_id', how='left') \
    .select('product_name', 'category_name')
  return product_category_pairs

In [ ]:
product_category_pairs = product_cat(product_df, product_category_df, category_df)
product_category_pairs.show()

+------------+--------------------+
|product_name|       category_name|
+------------+--------------------+
|         CPU|                NULL|
|      Laptop|         Electronics|
|     Monitor|         Electronics|
|     Monitor|         Peripherals|
|       Mouse|Computer Accessories|
|    Keyboard|Computer Accessories|
+------------+--------------------+



In [ ]:
spark.stop()